In [7]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

623

In [8]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | rikitea
Processing Record 2 of Set 1 | ponta do sol
Processing Record 3 of Set 1 | busselton
Processing Record 4 of Set 1 | isangel
Processing Record 5 of Set 1 | arraial do cabo
Processing Record 6 of Set 1 | punta arenas
Processing Record 7 of Set 1 | mataura
Processing Record 8 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 9 of Set 1 | saint-philippe
Processing Record 10 of Set 1 | pecos
Processing Record 11 of Set 1 | torbay
Processing Record 12 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 13 of Set 1 | cape town
Processing Record 14 of Set 1 | labuhan
Processing Record 15 of Set 1 | wurzburg
Processing Record 16 of Set 1 | terrasini
Processing Record 17 of Set 1 | mayumba
Processing Record 18 of Set 1 | lebu
Processing Record 19 of Set 1 | thompson
Processing Record 20 of Set 1 | port elizabeth
Processing Record 21 of Set 1 | nikolskoye
Processing

Processing Record 38 of Set 4 | olafsvik
Processing Record 39 of Set 4 | yerbogachen
Processing Record 40 of Set 4 | gurgan
City not found. Skipping...
Processing Record 41 of Set 4 | nova gradiska
Processing Record 42 of Set 4 | mahebourg
Processing Record 43 of Set 4 | manadhoo
Processing Record 44 of Set 4 | mirnyy
Processing Record 45 of Set 4 | antofagasta
Processing Record 46 of Set 4 | hambantota
Processing Record 47 of Set 4 | muscat
Processing Record 48 of Set 4 | ampanihy
Processing Record 49 of Set 4 | katsuura
Processing Record 50 of Set 4 | hemsedal
Processing Record 1 of Set 5 | acapulco
Processing Record 2 of Set 5 | carnarvon
Processing Record 3 of Set 5 | fevralsk
City not found. Skipping...
Processing Record 4 of Set 5 | sun valley
Processing Record 5 of Set 5 | mocuba
Processing Record 6 of Set 5 | meyungs
City not found. Skipping...
Processing Record 7 of Set 5 | tabriz
Processing Record 8 of Set 5 | alappuzha
Processing Record 9 of Set 5 | miramar
Processing Record

Processing Record 28 of Set 8 | isiro
Processing Record 29 of Set 8 | zhigansk
Processing Record 30 of Set 8 | peoria
Processing Record 31 of Set 8 | cordele
Processing Record 32 of Set 8 | hamilton
Processing Record 33 of Set 8 | pionerskiy
Processing Record 34 of Set 8 | hami
Processing Record 35 of Set 8 | kahului
Processing Record 36 of Set 8 | margate
Processing Record 37 of Set 8 | baykit
Processing Record 38 of Set 8 | kidodi
Processing Record 39 of Set 8 | yershov
Processing Record 40 of Set 8 | vardo
Processing Record 41 of Set 8 | miandoab
Processing Record 42 of Set 8 | shahrud
Processing Record 43 of Set 8 | lubao
Processing Record 44 of Set 8 | palencia
Processing Record 45 of Set 8 | tres barras
Processing Record 46 of Set 8 | kotka
Processing Record 47 of Set 8 | vila
Processing Record 48 of Set 8 | aitape
Processing Record 49 of Set 8 | cervo
Processing Record 50 of Set 8 | poum
Processing Record 1 of Set 9 | saint-paul
Processing Record 2 of Set 9 | kenai
Processing Re

Processing Record 19 of Set 12 | san andres
Processing Record 20 of Set 12 | hasaki
Processing Record 21 of Set 12 | amahai
Processing Record 22 of Set 12 | bakchar
Processing Record 23 of Set 12 | yellandu
Processing Record 24 of Set 12 | fort walton beach
Processing Record 25 of Set 12 | batagay
Processing Record 26 of Set 12 | maarianhamina
Processing Record 27 of Set 12 | evensk
Processing Record 28 of Set 12 | elat
Processing Record 29 of Set 12 | buqayq
City not found. Skipping...
Processing Record 30 of Set 12 | sal rei
Processing Record 31 of Set 12 | porec
Processing Record 32 of Set 12 | varniai
Processing Record 33 of Set 12 | bagdarin
Processing Record 34 of Set 12 | ulaangom
Processing Record 35 of Set 12 | oriximina
Processing Record 36 of Set 12 | alyangula
Processing Record 37 of Set 12 | skagastrond
City not found. Skipping...
Processing Record 38 of Set 12 | roald
Processing Record 39 of Set 12 | lata
Processing Record 40 of Set 12 | sijunjung
Processing Record 41 of 

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,77.31,78,69,17.45,PF,2022-05-03 22:28:54
1,Ponta Do Sol,32.6667,-17.1000,63.72,64,27,8.03,PT,2022-05-03 22:28:54
2,Busselton,-33.6500,115.3333,53.42,72,68,10.85,AU,2022-05-03 22:26:43
3,Isangel,-19.5500,169.2667,80.85,78,12,3.67,VU,2022-05-03 22:28:55
4,Arraial Do Cabo,-22.9661,-42.0278,78.66,78,0,6.91,BR,2022-05-03 22:28:56
5,Punta Arenas,-53.1500,-70.9167,42.91,81,40,34.52,CL,2022-05-03 22:24:18
6,Mataura,-46.1927,168.8643,52.48,73,11,2.39,NZ,2022-05-03 22:28:57
7,Saint-Philippe,-21.3585,55.7679,71.19,85,47,11.16,RE,2022-05-03 22:28:58
8,Pecos,31.4229,-103.4932,83.23,21,0,10.36,US,2022-05-03 22:28:58
9,Torbay,47.6666,-52.7314,44.28,88,75,11.50,CA,2022-05-03 22:28:59


In [13]:
# Reorder the columns
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Rikitea,PF,2022-05-03 22:28:54,-23.1203,-134.9692,77.31,78,69,17.45
1,Ponta Do Sol,PT,2022-05-03 22:28:54,32.6667,-17.1000,63.72,64,27,8.03
2,Busselton,AU,2022-05-03 22:26:43,-33.6500,115.3333,53.42,72,68,10.85
3,Isangel,VU,2022-05-03 22:28:55,-19.5500,169.2667,80.85,78,12,3.67
4,Arraial Do Cabo,BR,2022-05-03 22:28:56,-22.9661,-42.0278,78.66,78,0,6.91
...,...,...,...,...,...,...,...,...,...
565,Gashua,NG,2022-05-03 22:45:54,12.8710,11.0482,91.22,25,9,2.77
566,Sinop,TR,2022-05-03 22:45:54,41.7711,34.8709,49.24,97,82,2.28
567,Aldan,RU,2022-05-03 22:45:55,58.6031,125.3894,31.86,79,89,9.35
568,Joshimath,IN,2022-05-03 22:45:55,30.5667,79.5667,61.03,65,94,4.61


In [15]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")